In [1]:
!pip install opencv-python kociemba

In [2]:
!pip install numpy

In [3]:
pip install kociemba


Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install RubikTwoPhase

In [5]:
import cv2
import numpy as np
#import kociemba
import twophase.solver  as sv


loading conj_twist table...
loading conj_ud_edges table...
loading flipslice sym-tables...
loading corner sym-tables...
loading move_twist table...
loading move_flip table...
loading move_slice_sorted table...
loading move_u_edges table...
loading move_d_edges table...
loading move_ud_edges table...
loading move_corners table...
loading phase1_prun table...
loading phase2_prun table...
loading phase2_cornsliceprun table...
loading phase2_edgemerge table...


In [7]:
"""
Methodlogy:

1) 9 circles in the middle of webcam which will capture colors of cube
2) Take those circles, make it into hsv colorspace
3) 
3) 3d array will be used as input, just convert it to string using join() method

for when we have to look at color, we can hardcode logic that identifies each new face
based on the color of center piece, and then appends it accordingly to into 3d array

For Kociemba Module

U: Yellow Center Face
L: Blue Center Face
F: Red Center Face
R: Green Center Face
B: Blue Center Face
D: White Center Face
"""

'\nMethodlogy:\n\n1) 9 circles in the middle of webcam which will capture colors of cube\n2) Take those circles, make it into hsv colorspace\n3) \n3) 3d array will be used as input, just convert it to string using join() method\n\nfor when we have to look at color, we can hardcode logic that identifies each new face\nbased on the color of center piece, and then appends it accordingly to into 3d array\n\nFor Kociemba Module\n\nU: Yellow Center Face\nL: Blue Center Face\nF: Red Center Face\nR: Green Center Face\nB: Blue Center Face\nD: White Center Face\n'

In [14]:
import cv2
import numpy as np

# sets video capture to default webcam
# then we set the dimensions of the video capture

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)


# define circle positions
center_coordinate_x = 640
center_coordinate_y = 360  # Fixed typo here
offset_pixel_distance = 60

# calculated circle positions organized as a grid
circle_coordinates = [(center_coordinate_x - offset_pixel_distance, center_coordinate_y + offset_pixel_distance), (center_coordinate_x, center_coordinate_y + offset_pixel_distance), (center_coordinate_x + offset_pixel_distance, center_coordinate_y + offset_pixel_distance), # top row
                      (center_coordinate_x - offset_pixel_distance, center_coordinate_y), (center_coordinate_x, center_coordinate_y), (center_coordinate_x + offset_pixel_distance, center_coordinate_y), # middle row
                      (center_coordinate_x - offset_pixel_distance, center_coordinate_y - offset_pixel_distance), (center_coordinate_x, center_coordinate_y - offset_pixel_distance), (center_coordinate_x + offset_pixel_distance, center_coordinate_y - offset_pixel_distance)] # bottom row

# 3d array that holds the input cube state
input_cube_array = np.empty((6,3,3), dtype = str)

#compare hsv values and return color
def get_color(hue, sat, val):
    if sat < 50 and val > 180:
        return "W"
    elif (hue < 10 or hue > 170) and sat > 70:  # Adjusted red range and added saturation check
        return "R"
    elif 10 <= hue < 25 and sat > 70 and val > 50:  # Adjusted orange range and added saturation/value checks
        return "O"
    elif 25 <= hue < 35 and sat > 50:
        return "Y"
    elif 35 <= hue < 85 and sat > 40:
        return "G"
    elif 85 <= hue < 130 and sat > 40:
        return "B"
    else:
        return "U"  # unknown
    
def array_to_string(input_cube_array):

    twophase_output_string = ""
    
    DIMENSION_1 = 6
    DIMENSION_2 = 3
    DIMENSION_3 = 3

    for x in range(DIMENSION_1):
        for y in range(DIMENSION_2):
            for z in range(DIMENSION_3):
                twophase_output_string += input_cube_array[x][y][z]
    
    return twophase_output_string

# order of faces to scan
face_order = ["Yellow", "Green", "Red", "White", "Blue", "Orange"]
face_index = 0

# webcam logic
while True:

    # something new i learned
    # ret is a bool variable that returns true if the frame is available
    # frame is an image array vector captured based on default frames per second
    
    ret, frame = cap.read()

    # if webcame is not working
    if not ret:
        print("Failed to grab frame.")
        break

    # cv2.cv.flip(src, flipCode[, dst] )
    # 1 for dst which will mirror live stream

    frame = cv2.flip(frame, 1)

    # converting frame to hsv
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


    # now add the circles onto the frame
    # we need to loop through the coordinates and draw the circles
    # for j,k in my_list (stack overflow)
    for (c_x, c_y) in circle_coordinates:

        cv2.circle(frame, (c_x, c_y), 7, (25, 25, 25), 2)
        pixel_center_hsv = hsv_frame[c_y, c_x]  # Changed from frame to hsv_frame
        color = get_color(pixel_center_hsv[0], pixel_center_hsv[1], pixel_center_hsv[2])
        
        # display the current color to the right
        cv2.putText(frame, color, (c_x + 10, c_y), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0, 0, 0),2)

    # display current being displayed
    cv2.putText(frame, f"Scanning: {face_order[face_index]} Center Face", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Rubik's Cube Scanner", frame)


    key = cv2.waitKey(1) & 0xFF

    # logic for taking input for cube

    # if spacebar is pressed, capture the colors
    if key == ord(' '):
        for i, (c_x, c_y) in enumerate(circle_coordinates):
            print(f"Scanning face: {face_order[face_index]}")
            # Images in OpenCV are represented by NumPy arrays. To access a particular image pixel, all we need to do is pass in the (x, y)-coordinates as image[y, x]:
            pixel_center_hsv = hsv_frame[c_y,c_x]  #
            
            color = get_color(pixel_center_hsv[0], pixel_center_hsv[1], pixel_center_hsv[2])

            input_cube_array[face_index, i // 3, i % 3] = color
            
            print(f"Square {i+1}: {color}")

        face_index +=1
    
        # if all 6 sides are scanned, we are done
        if (face_index >=6):
            print("All faces scanned.")
            break


    # have a stop condition (click q for quit)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("Final Cube State: ")
print(input_cube_array)

def color_notation_to_kociemba_notation(str):

    if str == "Y":
        return "U"
    
    elif str == "B":
        return "L"
    
    elif str == "R":
        return "F"
    
    elif str == "G":
        return "R"
    
    elif str == "O":
        return "B"
    
    elif str == "W":
        return "D"
    
    else:
        return "?"


def array_to_string(input_cube_array):

    twophase_output_string = ""
    
    DIMENSION_1 = 6
    DIMENSION_2 = 3
    DIMENSION_3 = 3

    for x in range(DIMENSION_1):
        for y in range(DIMENSION_2):
            for z in range(DIMENSION_3):
                # we need to convert from colors to the actual sides
                str_temp = ""
                str_temp = color_notation_to_kociemba_notation(input_cube_array[x][y][z])
                twophase_output_string += str_temp
    
    return twophase_output_string

cube_string = array_to_string(input_cube_array)
print(f"Cube Input String: {cube_string}")

sv.solve(cube_string,19,2)

try: 
    solution = sv.solve(cube_string, 25, 5)
    print(f"Solution: {solution}")

except Exception as e:
    print(f"Error: {e}")



Failed to grab frame.
Final Cube State: 
[[['' '' '']
  ['' '' '']
  ['' '' '']]

 [['' '' '']
  ['' '' '']
  ['' '' '']]

 [['' '' '']
  ['' '' '']
  ['' '' '']]

 [['' '' '']
  ['' '' '']
  ['' '' '']]

 [['' '' '']
  ['' '' '']
  ['' '' '']]

 [['' '' '']
  ['' '' '']
  ['' '' '']]]
Cube Input String: ??????????????????????????????????????????????????????
Solution: Error: Cube definition string ?????????????????????????????????????????????????????? does not contain exactly 9 facelets of each color.


In [18]:
#https://pysource.com/2021/10/19/simple-color-recognition-with-opencv-and-python/
import cv2
import numpy as np


def nothing(x):
    pass

# Trackbar
cv2.namedWindow("frame")
cv2.createTrackbar("H", "frame", 0, 179, nothing)
cv2.createTrackbar("S", "frame", 255, 255, nothing)
cv2.createTrackbar("V", "frame", 255, 255, nothing)

img_hsv = np.zeros((250, 500, 3), np.uint8)

while True:
    h = cv2.getTrackbarPos("H", "frame")
    s = cv2.getTrackbarPos("S", "frame")
    v = cv2.getTrackbarPos("V", "frame")

    img_hsv[:] = (h, s, v)
    img_bgr = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

    cv2.imshow("frame", img_bgr)
    key = cv2.waitKey(1)
    if key == 27:
        break


cv2.destroyAllWindows()



error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2561: error: (-27:Null pointer) NULL window: 'frame' in function 'cvGetTrackbarPos'
